In [1]:
import time
import tensorflow as tf
import numpy as np
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from  tensorflow import keras
from  tensorflow.keras import layers
import base64
import cv2
import os
from tqdm import tqdm
from tensorflow.keras.utils import img_to_array
import re
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers import LeakyReLU
from keras.layers.core import Flatten, Dropout

SEED=1337

In [2]:
# initialize the data and labels
print("[INFO] loading images...")
images = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('train')))
random.seed(SEED)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, resize it to 64x64 pixels (the required input spatial dimensions of SmallVGGNet), 
    # and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (224, 224))   # we are not flattening our data for neural network, because it is convolutional
    images.append(image)

    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[1]
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
images = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

print('done')

[INFO] loading images...
done


In [3]:
# partition the data into 80% training and 20% validation
(trainX, valX, trainY, valY) = train_test_split(images, labels, test_size=0.2, random_state=SEED)

In [4]:
trainX.shape

(3584, 224, 224, 3)

In [5]:
lb = preprocessing.LabelBinarizer()
trainY = lb.fit_transform(trainY)
valY = lb.transform(valY)

In [6]:
input_shape = (trainX.shape[1], trainX.shape[2], trainX.shape[3])

In [7]:
from keras.applications import MobileNetV2

## Loading VGG16 model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False ## Not trainable weights

9406464/9406464 [==============================] - 1s 0us/step


In [8]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [9]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1024, activation='relu')
# dropput_layer_1 = layers.Dropout(0.25)
# dense_layer_2 = layers.Dense(512, activation='relu')
prediction_layer = layers.Dense(1, activation='sigmoid')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
#     dropput_layer_1,
#     dense_layer_2,
    prediction_layer
])

In [10]:
# model = tf.keras.models.load_model('saved_model/vgg_test.h5')

In [11]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

INIT_LR = 1e-4
EPOCHS = 5
BS = 32

opt = SGD(INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy"])

model.fit(aug.flow(trainX, trainY, batch_size=16), epochs=EPOCHS, validation_data=(valX, valY), batch_size=16, 
          callbacks=[keras.callbacks.EarlyStopping(patience=11, verbose=1, restore_best_weights=True),
                    keras.callbacks.ReduceLROnPlateau(factor=.5, patience=4, verbose=1)])

Epoch 1/5
224/224 [==============================] - 33s 119ms/step - loss: 0.6730 - binary_accuracy: 0.6454 - val_loss: 0.5270 - val_binary_accuracy: 0.7511 - lr: 1.0000e-04
Epoch 2/5
224/224 [==============================] - 25s 110ms/step - loss: 0.4774 - binary_accuracy: 0.7840 - val_loss: 0.4501 - val_binary_accuracy: 0.7969 - lr: 1.0000e-04
Epoch 3/5
224/224 [==============================] - 25s 109ms/step - loss: 0.4109 - binary_accuracy: 0.8161 - val_loss: 0.4124 - val_binary_accuracy: 0.8158 - lr: 1.0000e-04
Epoch 4/5
224/224 [==============================] - 24s 108ms/step - loss: 0.3575 - binary_accuracy: 0.8451 - val_loss: 0.3790 - val_binary_accuracy: 0.8393 - lr: 1.0000e-04
Epoch 5/5
224/224 [==============================] - 25s 110ms/step - loss: 0.3338 - binary_accuracy: 0.8521 - val_loss: 0.3604 - val_binary_accuracy: 0.8504 - lr: 1.0000e-04


In [13]:
model.save("saved_model/mobile_test.h5")

In [14]:
for layer in model.layers:
    layer.trainable = True

EPOCHS = 10

opt = Adam(1e-6)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy"])

model.fit(aug.flow(trainX, trainY, batch_size=16), epochs=EPOCHS, validation_data=(valX, valY), batch_size=16, 
          callbacks=[keras.callbacks.EarlyStopping(patience=11, verbose=1, restore_best_weights=True),
                    keras.callbacks.ReduceLROnPlateau(factor=.5, patience=4, verbose=1)])

Epoch 1/10
224/224 [==============================] - 32s 123ms/step - loss: 0.4989 - binary_accuracy: 0.7648 - val_loss: 0.3793 - val_binary_accuracy: 0.8371 - lr: 1.0000e-06
Epoch 2/10
224/224 [==============================] - 24s 108ms/step - loss: 0.4136 - binary_accuracy: 0.8119 - val_loss: 0.3390 - val_binary_accuracy: 0.8650 - lr: 1.0000e-06
Epoch 3/10
224/224 [==============================] - 24s 109ms/step - loss: 0.3805 - binary_accuracy: 0.8334 - val_loss: 0.3075 - val_binary_accuracy: 0.8672 - lr: 1.0000e-06
Epoch 4/10
224/224 [==============================] - 24s 109ms/step - loss: 0.3550 - binary_accuracy: 0.8449 - val_loss: 0.2960 - val_binary_accuracy: 0.8783 - lr: 1.0000e-06
Epoch 5/10
224/224 [==============================] - 25s 109ms/step - loss: 0.3311 - binary_accuracy: 0.8541 - val_loss: 0.2767 - val_binary_accuracy: 0.8940 - lr: 1.0000e-06
Epoch 6/10
224/224 [==============================] - 24s 109ms/step - loss: 0.3028 - binary_accuracy: 0.8672 - val_loss

In [15]:
model.save("saved_model/mobile_test.h5")

In [16]:
print("[INFO] loading test images...")
testData = []
testLabel = []

# grab the image paths and randomly shuffle them
testImagePaths = sorted(list(paths.list_images('test')))
random.seed(SEED)
random.shuffle(testImagePaths)

# loop over the input images
for imagePath in testImagePaths:
    # load the image, resize it to 64x64 pixels (the required input spatial dimensions of SmallVGGNet), 
    # and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (224, 224))   # we are not flattening our data for neural network, because it is convolutional
    testData.append(image)

    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[1]
    testLabel.append(label)

# scale the raw pixel intensities to the range [0, 1]
testData = np.array(testData, dtype="float") / 255.0
testLabel = np.array(testLabel)

print('done')

[INFO] loading test images...
done


In [3]:
testData.shape

(2046, 224, 224, 3)

In [17]:
testY = lb.transform(testLabel)

In [5]:
lb = preprocessing.LabelBinarizer()
testY = lb.fit_transform(testLabel)

### Evaluation

In [6]:
model = tf.keras.models.load_model('saved_model/vgg_test.h5')

In [18]:
predictions = model.predict(testData,  batch_size=8)

256/256 [==============================] - 3s 10ms/step


In [19]:
predictions =  (predictions>0.5)
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

           0       0.73      0.78      0.75      1271
           1       0.59      0.53      0.56       775

    accuracy                           0.68      2046
   macro avg       0.66      0.65      0.66      2046
weighted avg       0.68      0.68      0.68      2046



In [24]:
model.save("vgg_test_accuracy_70.h5")